In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

/workspace/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-13b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-13b-dream"

################################################################################
# QLoRA parameters
################################################################################

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [3]:
from datasets import Dataset
import json

with open('data_cube.json') as f:
    question_answer_set = json.load(f)
with open('eval_cube.json') as f:
    eval_set = json.load(f)

def format_prompts(example):
    formatting = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{question}
[/INST]
{answer}
""".strip()
    outputs = []

    for question, answer in zip(example["questions"], example["answers"]):
        outputs.append(formatting.format(question=question, answer=answer))

    return outputs


dataset = Dataset.from_dict(question_answer_set)
eval_dataset = Dataset.from_dict(eval_set)

In [4]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Your GPU supports bfloat16: accelerate training with bf16=True


In [5]:
def wandb_hp_space(trial):
    return {
        "method": "bayes",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "r": {
                "values": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            },
            "lora_alpha": {
                "values": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            },
            "lora_dropout": {"distribution": "uniform", "min": 0.0, "max": 0.15},
            "num_train_epochs": {"distribution": "int_uniform", "min": 1, "max": 8},
            "learning_rate": {
                "distribution": "log_uniform_values",
                "min": 1e-4,
                "max": 5e-3,
            }
        },
    }

# r = 32
# alpha = 64
# dropout = 0.1
# num_train_epochs = 12
# learning_rate = 1e-3

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.68s/it]
/workspace/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/workspace/env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.w

In [7]:
from peft.tuners.lora import LoraLayer
from bitsandbytes.nn import Linear4bit


def _replace_module(parent_module, child_name, new_module, old_module):
    setattr(parent_module, child_name, new_module)
    new_module.weight = old_module.weight


def _get_submodules(model, key):
    parent = model.get_submodule(".".join(key.split(".")[:-1]))
    target_name = key.split(".")[-1]
    target = model.get_submodule(key)
    return parent, target, target_name


def unload_lora(model):
    key_list = [key for key, _ in model.named_modules() if "lora" not in key]

    for key in key_list:
        try:
            parent, target, target_name = _get_submodules(model, key)
        except:
            continue
        if isinstance(target, LoraLayer):
            new_module = Linear4bit(
                target.in_features, target.out_features, bias=False)
            _replace_module(parent, target_name, new_module, target)

In [8]:
def model_init(trial):
    print("LOADING NEW MODEL")
    unload_lora(model)
    peft_config = LoraConfig(
        bias="none",
        task_type="CAUSAL_LM",
        r=trial['r'],
        lora_alpha=trial['lora_alpha'],
        lora_dropout=trial['lora_dropout'],
    )
    peft_model = get_peft_model(model, peft_config)
    return peft_model

In [9]:
peft_model = model_init({
    'r': 16,
    'lora_alpha': 64,
    'lora_dropout': 0.1
})

LOADING NEW MODEL


In [10]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=peft_model,
    model_init=model_init,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    max_seq_length=max_seq_length,
    formatting_func=format_prompts,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map: 100%|██████████| 1/1 [00:00<00:00, 140.09 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/workspace/env/lib/python3.10/site-packages/transformers/trainer.py:363: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(


In [11]:
def compute_objective(metrics):
    # return metrics["eval_loss"]
    print(metrics)
    return metrics["eval_loss"]

trainer.hyperparameter_search(
    direction="minimize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=10000,
    compute_objective=compute_objective,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: xj4s8olz
Sweep URL: https://wandb.ai/decycle/uncategorized/sweeps/xj4s8olz


wandb: Agent Starting Run: rz8vjeo5 with config:
wandb: 	learning_rate: 0.00013045173249711604
wandb: 	lora_alpha: 8
wandb: 	lora_dropout: 0.07226152092135012
wandb: 	num_train_epochs: 2
wandb: 	r: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: decycle. Use `wandb login --relogin` to force relogin


Trying to set lora_alpha in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set r in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


LOADING NEW MODEL


...

Trying to set lora_alpha in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set r in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


LOADING NEW MODEL


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Step,Training Loss
